#Conversation Summarizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pwd

In [ ]:
!ls
%cd drive/MyDrive/splore_project
!pwd
!ls

In [ ]:
import pandas as pd
import numpy as np

###Reading dataset

In [ ]:
dataset = pd.read_csv("topical_chat.csv")
dataset.drop(dataset[dataset['message'].isnull()].index , inplace=True)
dataset

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpful...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper
...,...,...,...
188373,8628,"Wow, it does not seem like that long. Since I...",Surprised
188374,8628,"I havent seen that episode, I might google it...",Curious to dive deeper
188375,8628,I don't think I have either. That's an insane ...,Curious to dive deeper
188376,8628,"I did, my little brother used to love Thomas ...",Happy


###Merging all messages of a conversation into a single message

In [ ]:
new_dataset = pd.DataFrame([{'conv_id': id, 'dialogue': ' \n '.join(dataset[dataset.conversation_id==id]['message'].values.tolist())} for id in dataset.conversation_id.unique()[:100]])
# new_dataset.to_csv('./merged_chats.csv')
new_dataset

,conv_id,dialogue
0,1,Are you a fan of Google or Microsoft? \n Both ...
1,2,do you like dance? \n Yes I do. Did you know ...
2,3,Hey what's up do use Google very often?I reall...
3,4,Hi! do you like to dance? \n I love to dance ...
4,5,do you like dance? \n I love it. Did you know ...
...,...,...
95,96,"Dogs or cats? \n Hi! I have two cats, but wan..."
96,97,Crazy that early humans used to have to battle...
97,98,"Hi, Do you like to cook? \n I love to cook. j..."
98,99,Hi do you like tennis? \n Oh yeah! I pick up a...


In [ ]:
del dataset

###Using HuggingFace transformers

In [ ]:
!pip install transformers huggingface_hub
!pip install accelerate -U
!pip install datasets

In [ ]:
import transformers
from transformers import pipeline
import torch.nn as nn

###Using summarization pipeline for pretrained T5-base (Text-To-Text Transfer Transformer) model

In [ ]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

In [18]:
all_summ = []
for i in range(len(new_dataset)):
  summary = summarizer(new_dataset.dialogue[i], min_length=10, max_length=50)
  all_summ.append({'conv_id': new_dataset.conv_id[i], 'summary': summary[0]['summary_text']})
  print(summary[0]['summary_text'])

google is leading the alphabet subsidiary and will continue to be the Umbrella company for Alphabet internet interest . did you know that google had hundreds of live goats to cut the grass in the past?


Token indices sequence length is longer than the specified maximum sequence length for this model (521 > 512). Running this sequence through the model will result in indexing errors


do you like dancing? yes I do . did you know Bruce Lee was a cha cha dancer? yes he did . do you know that he popularized many phrases Yes .
i think everyone must use google daily! its become ingrained in every day life . i also want to go to google Plex to see the goats who mow their lawn by eating it .
bob greene: did you know that Bruce Lee was a great dancer? he says he danced ballet in high school and was the mouse king in the nutcracker . greene says that's
did you know Bruce Lee was a dancer? yes he even won a cha cha championship in 1958 Yeah . do you like SHakespeare? i love his work .
a google search takes more computing power than sending astronauts tot he moon . do you use gmail? yes, you do . have you ever typed askew into google? no I have .
the largest comic book market is in Japan. I believe it. my son loves to read those mangas. I would love to go there. . the US market in the US and Canada is $1.09 billion!
the universe is amazing and ever evolving . there is even pla

In [19]:
all_summ_df = pd.DataFrame(all_summ)
all_summ_df.to_csv('./100_summaries.csv')
all_summ_df

,conv_id,summary
0,1,google is leading the alphabet subsidiary and ...
1,2,do you like dancing? yes I do . did you know B...
2,3,i think everyone must use google daily! its be...
3,4,bob greene: did you know that Bruce Lee was a ...
4,5,did you know Bruce Lee was a dancer? yes he ev...
...,...,...
95,96,"i like dogs more, but dont mind cats! i have t..."
96,97,i think it's really surprising humans are the ...
97,98,the oldest goldfish lived to be 43 years old W...
98,99,tennis has an interesting history . until 1978...
